In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from gensim.models import Word2Vec
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss, mean_squared_log_error
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
import tqdm
import sys
import os
import gc
import argparse
import warnings
import lightgbm as lgb
from lightgbm import early_stopping
from lightgbm import log_evaluation
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif']=['SimHei']

In [2]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## 读取数据

In [3]:
jy_df=pd.read_csv('data/账户交易信息.csv')
jy_df.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']
jt_df=pd.read_csv('data/账户静态信息.csv')
jt_df.columns = ['账户代号','开户日期','开户行代号','客户性别','年龄']
train_label=pd.read_csv('data/训练集标签.csv')
train_label.columns = ['账户代号','label']
test_df=pd.read_csv('data/test_dataset.csv')
test_df.columns = ['账户代号']

In [4]:
df_all=pd.concat([train_label,test_df])

## 抹账数据预处理

In [5]:
# 抹账 刚刚好都能抹干净，总共有2003条负数的交易记录

jy_mz_df=jy_df[jy_df['账户代号'].isin(jy_df[jy_df['交易金额']<0]['账户代号'].unique())].sort_values(by=['账户代号','交易日期','交易时间'],ascending=True)
jy_mz_df.index=range(len(jy_mz_df))
#待剔除数据
tc_idx=set()
for idx in jy_mz_df[jy_mz_df['交易金额']<0].index:
    for d in range(1,5):
        #print(idx)
        if -jy_mz_df.loc[idx,'交易金额']==jy_mz_df.loc[idx-d,'交易金额'] and jy_mz_df.loc[idx,'账户代号']==jy_mz_df.loc[idx-d,'账户代号']\
            and jy_mz_df.loc[idx,'对方账号']==jy_mz_df.loc[idx-d,'对方账号']:
                tc_idx.add(idx)
                tc_idx.add(idx-d)
                break
tc_idx=list(tc_idx)
jy_clean_df=jy_mz_df[~jy_mz_df.index.isin(tc_idx)]
jy_clean_df=pd.concat([jy_clean_df,jy_df[jy_df['交易金额']>=0]])

## 时间特征

In [6]:
def 获取时间特征(df, col):
    
    df_copy = df.copy()
    prefix = col + "_"
    df_copy['new_'+col] = df_copy[col].astype(str)
    
    col = 'new_'+col
    
    df_copy[col] = pd.to_datetime(df_copy[col], format='%Y-%m-%d')

    df_copy[prefix + 'day'] = df_copy[col].dt.day
    df_copy[prefix + 'month'] = df_copy[col].dt.month
    df_copy[prefix + 'weekofyear'] = df_copy[col].dt.weekofyear
    df_copy[prefix + 'dayofyear'] = df_copy[col].dt.dayofyear
    df_copy[prefix + 'dayofweek'] = df_copy[col].dt.dayofweek
    df_copy[prefix + 'is_wknd'] = (df_copy[col].dt.dayofweek + 1) // 6
    df_copy[prefix + 'is_month_start'] = df_copy[col].dt.is_month_start.astype(int)
    df_copy[prefix + 'is_month_end'] = df_copy[col].dt.is_month_end.astype(int)
    del df_copy[col]
    
    df_copy[prefix + 'hour'] = df_copy['交易时间'].apply(lambda x:int(x.split(':')[0]))
    df_copy[prefix + 'minu'] = df_copy['交易时间'].apply(lambda x:int(x.split(':')[1]))
    df_copy[prefix + 'date'] = df_copy['交易时间'].apply(lambda x:int(x.split(':')[0])*60 + int(x.split(':')[1]))
    df_copy[prefix + 'ismidnight']=df_copy[prefix + 'hour'].apply(lambda x: 1 if x>=11 and x<=4 else 0)
    df_copy[prefix + 'isworktime']=df_copy[prefix + 'hour'].apply(lambda x: 1 if x>=8 and x<=18 else 0)
    del df_copy['交易时间']
    return df_copy


In [7]:
jy_clean_df = 获取时间特征(jy_clean_df, "交易日期")
# jy_df = 获取时间特征(jy_df, "交易日期")

In [8]:
jy_clean_df.head(5)

,交易流水序号,账户代号,对方账号,借贷标志,交易金额,交易余额,对方行号,交易日期,交易渠道,摘要代号,对方名称长度,交易日期_day,交易日期_month,交易日期_weekofyear,交易日期_dayofyear,交易日期_dayofweek,交易日期_is_wknd,交易日期_is_month_start,交易日期_is_month_end,交易日期_hour,交易日期_minu,交易日期_date,交易日期_ismidnight,交易日期_isworktime
0,848365FF4D60AF99E1938BE501EC6A51,00037295453A928A,14BEFED1370B730A,1,2590.17,2652.09,834E1F06,2020-03-02,E96ED478,C22ABFA3,6,2,3,10,62,0,0,0,0,21,38,1298,0,0
1,93AF2249D1D16CF16CABA358563FDC19,00037295453A928A,2D5FC4454465D9F3,0,1200.00,1452.09,583BDDDA,2020-03-02,091D584F,A97DA629,6,2,3,10,62,0,0,0,0,21,56,1316,0,0
2,AD65DD2ADCE3B705223A927D9B4897D6,00037295453A928A,14BEFED1370B730A,1,2077.15,3529.24,834E1F06,2020-03-03,E96ED478,C22ABFA3,6,3,3,10,63,1,0,0,0,22,26,1346,0,0
3,6EB57994413EDE45AEFF1D90DB2D1C87,00037295453A928A,EDE31EF0F69DC342,0,1789.00,1740.24,469F7281,2020-03-03,E96ED478,4E0CB6FB,12,3,3,10,63,1,0,0,0,22,27,1347,0,0
4,5B3BF52231268A9B19CC20357B9F98ED,00037295453A928A,D98BD9FA9D006723,1,3000.00,4740.24,469F7281,2020-03-03,E96ED478,C22ABFA3,33,3,3,10,63,1,0,0,0,23,0,1380,0,0


## 按时间特征聚合

In [30]:
%%time
def 按时间特征聚合(df1_, df2_):
    df1 = df1_.copy() #交易流水 
    df2 = df2_.copy() #全体客户
    time_cols = [f for f in df1.columns if '交易日期_' in f]
    
    # np.ptp 求最大值和最小值差值

    agg_func = {
        '对方账号': ['nunique','count'],
        '对方行号': ['nunique'],
        '交易渠道': ['nunique'],
        '摘要代号': ['nunique'],
        '交易金额': ['sum','mean','max','min','std',np.ptp],
        '交易余额': ['sum','mean','max','min','std',np.ptp],
        '对方名称长度': ['mean','max','min','std',np.ptp],
    }
    
    for col in time_cols:
        agg_func[col] = ['mean','min','max',np.ptp]
    
    # 交易流水 按照 账户代号 groupby ，然后求各种信息，最后再reset_index() ;
    agg_df0 = df1.groupby(['账户代号']).agg(agg_func).reset_index()
    agg_df0.columns = ['账户代号'] + ['账户代号_借贷标志流量_' + '_'.join(f).strip() for f in agg_df0.columns.values if f[0] not in ['账户代号']]
    
    agg_df1 = df1[df1['借贷标志']==0].groupby(['账户代号']).agg(agg_func).reset_index()
    agg_df1.columns = ['账户代号'] + ['账户代号_借贷标志0_' + '_'.join(f).strip() for f in agg_df1.columns.values if f[0] not in ['账户代号']]
    
    
    agg_df2 = df1[df1['借贷标志']==1].groupby(['账户代号']).agg(agg_func).reset_index()
    agg_df2.columns = ['账户代号'] + ['账户代号_借贷标志1_' + '_'.join(f).strip() for f in agg_df2.columns.values if f[0] not in ['账户代号']]
    
    
    
    df_all=pd.merge(df2,agg_df0,on='账户代号',how='left')
    df_all=pd.merge(df_all,agg_df1,on='账户代号',how='left')
    df_all=pd.merge(df_all,agg_df2,on='账户代号',how='left')
    
    for idx,c2 in enumerate(agg_df2.columns):
        if idx>=1:
            c1=agg_df1.columns[idx]
            df_all[c2+'-'+c1]=df_all[c2]-df_all[c1]
        
    return df_all

df_all = 按时间特征聚合(jy_clean_df, df_all)

Wall time: 1min 5s


In [31]:
df_all.head()

,账户代号,label,账户代号_借贷标志流量_对方账号_nunique,账户代号_借贷标志流量_对方账号_count,账户代号_借贷标志流量_对方行号_nunique,账户代号_借贷标志流量_交易渠道_nunique,账户代号_借贷标志流量_摘要代号_nunique,账户代号_借贷标志流量_交易金额_sum,账户代号_借贷标志流量_交易金额_mean,账户代号_借贷标志流量_交易金额_max,账户代号_借贷标志流量_交易金额_min,账户代号_借贷标志流量_交易金额_std,账户代号_借贷标志流量_交易金额_ptp,账户代号_借贷标志流量_交易余额_sum,账户代号_借贷标志流量_交易余额_mean,账户代号_借贷标志流量_交易余额_max,账户代号_借贷标志流量_交易余额_min,账户代号_借贷标志流量_交易余额_std,账户代号_借贷标志流量_交易余额_ptp,账户代号_借贷标志流量_对方名称长度_mean,账户代号_借贷标志流量_对方名称长度_max,账户代号_借贷标志流量_对方名称长度_min,账户代号_借贷标志流量_对方名称长度_std,账户代号_借贷标志流量_对方名称长度_ptp,账户代号_借贷标志流量_交易日期_day_mean,账户代号_借贷标志流量_交易日期_day_min,账户代号_借贷标志流量_交易日期_day_max,账户代号_借贷标志流量_交易日期_day_ptp,账户代号_借贷标志流量_交易日期_month_mean,账户代号_借贷标志流量_交易日期_month_min,账户代号_借贷标志流量_交易日期_month_max,账户代号_借贷标志流量_交易日期_month_ptp,账户代号_借贷标志流量_交易日期_weekofyear_mean,账户代号_借贷标志流量_交易日期_weekofyear_min,账户代号_借贷标志流量_交易日期_weekofyear_max,账户代号_借贷标志流量_交易日期_weekofyear_ptp,账户代号_借贷标志流量_交易日期_dayofyear_mean,账户代号_借贷标志流量_交易日期_dayofyear_min,账户代号_借贷标志流量_交易日期_dayofyear_max,账户代号_借贷标志流量_交易日期_dayofyear_ptp,账户代号_借贷标志流量_交易日期_dayofweek_mean,账户代号_借贷标志流量_交易日期_dayofweek_min,账户代号_借贷标志流量_交易日期_dayofweek_max,账户代号_借贷标志流量_交易日期_dayofweek_ptp,账户代号_借贷标志流量_交易日期_is_wknd_mean,账户代号_借贷标志流量_交易日期_is_wknd_min,账户代号_借贷标志流量_交易日期_is_wknd_max,账户代号_借贷标志流量_交易日期_is_wknd_ptp,账户代号_借贷标志流量_交易日期_is_month_start_mean,账户代号_借贷标志流量_交易日期_is_month_start_min,账户代号_借贷标志流量_交易日期_is_month_start_max,账户代号_借贷标志流量_交易日期_is_month_start_ptp,账户代号_借贷标志流量_交易日期_is_month_end_mean,账户代号_借贷标志流量_交易日期_is_month_end_min,账户代号_借贷标志流量_交易日期_is_month_end_max,账户代号_借贷标志流量_交易日期_is_month_end_ptp,账户代号_借贷标志流量_交易日期_hour_mean,账户代号_借贷标志流量_交易日期_hour_min,账户代号_借贷标志流量_交易日期_hour_max,账户代号_借贷标志流量_交易日期_hour_ptp,账户代号_借贷标志流量_交易日期_minu_mean,账户代号_借贷标志流量_交易日期_minu_min,账户代号_借贷标志流量_交易日期_minu_max,账户代号_借贷标志流量_交易日期_minu_ptp,账户代号_借贷标志流量_交易日期_date_mean,账户代号_借贷标志流量_交易日期_date_min,账户代号_借贷标志流量_交易日期_date_max,账户代号_借贷标志流量_交易日期_date_ptp,账户代号_借贷标志流量_交易日期_ismidnight_mean,账户代号_借贷标志流量_交易日期_ismidnight_min,账户代号_借贷标志流量_交易日期_ismidnight_max,账户代号_借贷标志流量_交易日期_ismidnight_ptp,账户代号_借贷标志流量_交易日期_isworktime_mean,账户代号_借贷标志流量_交易日期_isworktime_min,账户代号_借贷标志流量_交易日期_isworktime_max,账户代号_借贷标志流量_交易日期_isworktime_ptp,账户代号_借贷标志0_对方账号_nunique,账户代号_借贷标志0_对方账号_count,账户代号_借贷标志0_对方行号_nunique,账户代号_借贷标志0_交易渠道_nunique,账户代号_借贷标志0_摘要代号_nunique,账户代号_借贷标志0_交易金额_sum,账户代号_借贷标志0_交易金额_mean,账户代号_借贷标志0_交易金额_max,账户代号_借贷标志0_交易金额_min,账户代号_借贷标志0_交易金额_std,账户代号_借贷标志0_交易金额_ptp,账户代号_借贷标志0_交易余额_sum,账户代号_借贷标志0_交易余额_mean,账户代号_借贷标志0_交易余额_max,账户代号_借贷标志0_交易余额_min,账户代号_借贷标志0_交易余额_std,账户代号_借贷标志0_交易余额_ptp,账户代号_借贷标志0_对方名称长度_mean,账户代号_借贷标志0_对方名称长度_max,账户代号_借贷标志0_对方名称长度_min,账户代号_借贷标志0_对方名称长度_std,账户代号_借贷标志0_对方名称长度_ptp,账户代号_借贷标志0_交易日期_day_mean,账户代号_借贷标志0_交易日期_day_min,账户代号_借贷标志0_交易日期_day_max,账户代号_借贷标志0_交易日期_day_ptp,账户代号_借贷标志0_交易日期_month_mean,账户代号_借贷标志0_交易日期_month_min,账户代号_借贷标志0_交易日期_month_max,账户代号_借贷标志0_交易日期_month_ptp,账户代号_借贷标志0_交易日期_weekofyear_mean,账户代号_借贷标志0_交易日期_weekofyear_min,账户代号_借贷标志0_交易日期_weekofyear_max,账户代号_借贷标志0_交易日期_weekofyear_ptp,账户代号_借贷标志0_交易日期_dayofyear_mean,账户代号_借贷标志0_交易日期_dayofyear_min,账户代号_借贷标志0_交易日期_dayofyear_max,账户代号_借贷标志0_交易日期_dayofyear_ptp,账户代号_借贷标志0_交易日期_dayofweek_mean,账户代号_借贷标志0_交易日期_dayofweek_min,账户代号_借贷标志0_交易日期_dayofweek_max,账户代号_借贷标志0_交易日期_dayofweek_ptp,账户代号_借贷标志0_交易日期_is_wknd_mean,账户代号_借贷标志0_交易日期_is_wknd_min,账户代号_借贷标志0_交易日期_is_wknd_max,账户代号_借贷标志0_交易日期_is_wknd_ptp,账户代号_借贷标志0_交易日期_is_month_start_mean,账户代号_借贷标志0_交易日期_is_month_start_min,账户代号_借贷标志0_交易日期_is_month_start_max,账户代号_借贷标志0_交易日期_is_month_start_ptp,账户代号_借贷标志0_交易日期_is_month_end_mean,账户代号_借贷标志0_交易日期_is_month_end_min,账户代号_借贷标志0_交易日期_is_month_end_max,账户代号_借贷标志0_交易日期_is_month_end_ptp,账户代号_借贷标志0_交易日期_hour_mean,账户代号_借贷标志0_交易日期_hour_min,账户代号_借贷标志0_交易日期_hour_max,账户代号_借贷标志0_交易日期_hour_ptp,账户代号_借贷标志0_交易日期_minu_mean,账户代号_借贷标志0_交易日期_minu_min,账户代号_借贷标志0_交易日期_minu_max,账户代号_借贷标志0_交易日期_minu_ptp,账户代号_借贷标志0_交易日期_date_mean,账户代号_借贷标志0_交易日期_date_min,账户代号_借贷标志0_交易日期_date_max,账户代号_借贷标志0_交易日期_date_ptp,账户代号_借贷标志0_交易日期_ismidnight_mean,账户代号_借贷标志0_交易日期_ismidnight_min,账户代号_借贷标志0_交易日期_ismidnight_max,账户代号_借贷标志0_交易日期_ismidnight_ptp,账户代号_借贷标志0_交易日期_isworktime_mean,账户代号_借贷标志0_交易日期_isworktime_min,账户代号_借贷标志0_交易日期_

## 时间滑窗

In [33]:
%%time
def 时间滑窗特征聚合(df1_,df2_):
    # df1 交易流水
    # df2 用户主档
    df1 = df1_.copy() #交易流水 
    df2 = df2_.copy()
    df1_maxrq=df1.groupby(['账户代号'])['交易日期'].max().reset_index()
    df1_maxrq.columns=['账户代号','最近交易日期']
    df1=pd.merge(df1,df1_maxrq,on='账户代号',how='left') 
    
    time_cols = [f for f in df1.columns if '交易日期_' in f]

    agg_func = {
        '对方账号': ['nunique','count'],
        '对方行号': ['nunique'],
        '交易渠道': ['nunique'],
        '摘要代号': ['nunique'],
        '交易金额': ['sum','mean','max','min','std',np.ptp],
        '交易余额': ['sum','mean','max','min','std',np.ptp],
        '对方名称长度': ['mean','max','min','std',np.ptp],
    }
    
    for col in time_cols:
        agg_func[col] = ['mean','min','max',np.ptp]
    
    df1['date_diff']=(pd.to_datetime(df1['最近交易日期'],format='%Y-%m-%d')-pd.to_datetime(df1['交易日期'],format='%Y-%m-%d')).dt.days
    df_all=df2
    for i in [15,30,60,90]:
        agg_df0 = df1.groupby(['账户代号']).agg(agg_func).reset_index()
        agg_df0.columns = ['账户代号'] + ['账户代号_借贷标志不区分_'+str(i)+'天以内' + '_'.join(f).strip() for f in agg_df0.columns.values if f[0] not in ['账户代号']]

        agg_df1 = df1[df1['借贷标志']==0].groupby(['账户代号']).agg(agg_func).reset_index()
        agg_df1.columns = ['账户代号'] + ['账户代号_借贷标志0_'+str(i)+'天以内' + '_'.join(f).strip() for f in agg_df1.columns.values if f[0] not in ['账户代号']]


        agg_df2 = df1[df1['借贷标志']==1].groupby(['账户代号']).agg(agg_func).reset_index()
        agg_df2.columns = ['账户代号'] + ['账户代号_借贷标志1_'+str(i)+'天以内' + '_'.join(f).strip() for f in agg_df2.columns.values if f[0] not in ['账户代号']]
        
        
    
        df_all=pd.merge(df_all,agg_df0,on='账户代号',how='left')
        df_all=pd.merge(df_all,agg_df1,on='账户代号',how='left')
        df_all=pd.merge(df_all,agg_df2,on='账户代号',how='left')
        
        for idx,c2 in enumerate(agg_df2.columns):
            if idx>=1:
                c1=agg_df1.columns[idx]
                df_all[c2+'-'+c1]=df_all[c2]-df_all[c1]
        
    return df_all

df_sjhc_clean = 时间滑窗特征聚合(jy_clean_df,df_all['账户代号'])
df_sjhc_jy = 时间滑窗特征聚合(jy_df,df_all['账户代号'])

KeyboardInterrupt: 

## 补充

In [ ]:
def 补充特征(df1_,df2_):
    df1 = df1_.copy() #交易流水 
    df2 = df2_.copy()
    

    
    #每日交易金额汇总
    df1_sumbyday_0=df1[df1['借贷标志']==0].groupby(['账户代号','交易日期'])['交易金额'].sum().reset_index()
    df1_sumbyday_0.columns=['账户代号','交易日期','单日0方向交易金额']
    df1_sumbyday_1=df1[df1['借贷标志']==1].groupby(['账户代号','交易日期'])['交易金额'].sum().reset_index()
    df1_sumbyday_1.columns=['账户代号','交易日期','单日1方向交易金额']
    
    
    df1_sumbyday=pd.merge(df2,df1_sumbyday_0,on='账户代号',how='left')
    df1_sumbyday=pd.merge(df1_sumbyday,df1_sumbyday_1,on=['账户代号','交易日期'],how='left')
    
    df1_sumbyday['单日净交易金额']=df1_sumbyday['单日1方向交易金额']-df1_sumbyday['单日0方向交易金额']
    
    #最大交易金额日期
    df1_sumbyday_maxje=df1_sumbyday.groupby(['账户代号'])['单日0方向交易金额','单日1方向交易金额','单日净交易金额'].max().reset_index()
    df1_sumbyday_maxje.columns=['账户代号','单日0方向最大交易金额','单日1方向最大交易金额','单日最大净交易金额']
    
    
    df_maxrq_byday=pd.DataFrame(df2)
    c1_list=['单日0方向交易金额','单日1方向交易金额','单日净交易金额']
    for idx,c in enumerate(['单日0方向最大交易金额','单日1方向最大交易金额','单日最大净交易金额']):
        c1=c1_list[idx]
        df1_sumbyday_t0=df1_sumbyday.merge(df1_sumbyday_maxje,left_on=['账户代号',c1],right_on=['账户代号',c],how='inner')
        df1_sumbyday_t0=df1_sumbyday_t0.groupby(['账户代号'])['交易日期'].min().reset_index()
        df1_sumbyday_t0.columns=['账户代号',c+'_'+'交易日期']
        df_maxrq_byday=pd.merge(df_maxrq_byday,df1_sumbyday_t0,on='账户代号',how='left')

    

    ##最近交易日期
    df1_maxrq=df1.groupby(['账户代号'])['交易日期'].max().reset_index()
    df1_maxrq.columns=['账户代号','最近交易日期']
    
    df_maxrq_byday=df_maxrq_byday.merge(df1_maxrq,on='账户代号',how='left')
    
    for c in ['单日0方向最大交易金额_交易日期','单日1方向最大交易金额_交易日期','单日最大净交易金额_交易日期']:
        df_maxrq_byday[c+'距离最近交易_天数']=(pd.to_datetime(df_maxrq_byday['最近交易日期'],format='%Y-%m-%d')-pd.to_datetime(df_maxrq_byday[c],format='%Y-%m-%d')).dt.days
    df_all=df_maxrq_byday
    df_all.drop(['最近交易日期','单日0方向最大交易金额_交易日期','单日1方向最大交易金额_交易日期','单日最大净交易金额_交易日期'],axis=1,inplace=True)
    #抹账笔数
    df_mz=df1[df1['交易金额']<0]
    
    time_cols = [f for f in df1.columns if '交易日期_' in f]
    
    agg_func = {
        '对方账号': ['nunique','count'],
        '交易金额': ['sum','mean','max','min','std',np.ptp],
        '对方名称长度': ['mean','max','min','std',np.ptp],
    }
    
    for col in time_cols:
        agg_func[col] = ['mean','min','max',np.ptp]

    agg_df1 = df_mz[df_mz['借贷标志']==0].groupby(['账户代号']).agg(agg_func).reset_index()
    agg_df1.columns = ['账户代号'] + ['账户代号_借贷标志0_抹账' + '_'.join(f).strip() for f in agg_df1.columns.values if f[0] not in ['账户代号']]

    agg_df2 = df_mz[df_mz['借贷标志']==1].groupby(['账户代号']).agg(agg_func).reset_index()
    agg_df2.columns = ['账户代号'] + ['账户代号_借贷标志1_抹账' + '_'.join(f).strip() for f in agg_df2.columns.values if f[0] not in ['账户代号']]
    
    df_all=pd.merge(df_all,agg_df1,on='账户代号',how='left')
    df_all=pd.merge(df_all,agg_df2,on='账户代号',how='left')
    return df_all


df_bc_jy = 补充特征(jy_df,df_all['账户代号'])

## 合并账户基本信息

In [ ]:
%%time
def 账户基础信息(df1_):
    df1 = df1_.copy() #基本信息
    
    df1['开户日期']  = pd.to_datetime(df1['开户日期'], format='%Y-%m-%d')
    df1['开户日期_year']  = df1['开户日期'].dt.year
    df1['开户日期_month'] = df1['开户日期'].dt.month
    df1['开户日期_day']   = df1['开户日期'].dt.day
    del df1['开户日期']
    return df1

jt_df = 账户基础信息(jt_df)

## 数据合并

In [ ]:
df_all=df_all.merge(jt_df,on='账户代号',how='left')
df_all=df_all.merge(df_sjhc_clean,on='账户代号',how='left')
df_all=df_all.merge(df_sjhc_jy,on='账户代号',how='left')
df_all=df_all.merge(df_bc_jy,on='账户代号',how='left')


In [ ]:
#catboost编码
def cat_encode(df_train_,df_test_):
    df_train=df_train_.copy()
    df_test=df_test_.copy()
    cat_feature_lists=['开户行代号']
    for cat_feature in cat_feature_lists:
        target_enc = ce.CatBoostEncoder(cols=cat_feature,a=200)
        target_enc.fit(df_train[cat_feature], df_train['label'])
    #Transform the features, rename columns with _cb suffix, and join to dataframe
        df_train[cat_feature] = target_enc.transform(df_train[cat_feature]).add_suffix('_cb')
        df_test[cat_feature] = target_enc.transform(df_test[cat_feature]).add_suffix('_cb')
        df_train[cat_feature]
        df_test[cat_feature]
    return df_train,df_test

train_label,test_label = cat_encode(df_all[:len(train_label)],df_all[len(train_label):])

In [ ]:
def corr_filter(train_data,corr_threshold):
    corr_data=train_data.corr()
    corr_data=corr_data.where(np.triu(np.ones(corr_data.shape),k=1).astype(np.bool))
    high_corr=[column for column in corr_data.columns if any(corr_data[column].abs()>corr_threshold)]
    result=[c for c in train_data.columns if c not in high_corr]
    print("筛选后特征:",len(result))
    return result


In [ ]:
cols = [f for f in train_label.columns if f not in ['账户代号','label']]
cols =corr_filter(train_label[cols],0.9)

In [ ]:
def 对抗验证获取数据(df_train,df_test,threshold):
    # 定义新的Y
    
    
    df_train['Is_Test'] = 0
    df_test['Is_Test'] = 1
    #print(df_train.columns)
    #print(df_test.columns)
    # 将 Train 和 Test 合成一个数据集。
    df_adv = pd.concat([df_train, df_test])
    #print(df_adv.columns)
    #catecol_list_index=[list(df_train.columns).index(c) for c in df_train.select_dtypes(include=['category']).columns]
    # 通过抗验证中的模型，得到各个样本属于测试集的概率


    
    model_adv =  lgb.LGBMClassifier()
    model_adv.fit(df_adv.drop(['label','Is_Test'], axis=1), df_adv.loc[:, 'Is_Test'])
    preds_adv = model_adv.predict_proba(df_adv.drop(['label','Is_Test'], axis=1))[:, 1]
    
    df_train_copy = df_train.copy()
    df_train_copy['is_test_prob'] = preds_adv[:len(df_train)]

    # 根据概率排序
    df_train_copy = df_train_copy.sort_values('is_test_prob').reset_index(drop=True)

    # 将概率最大的20%作为验证集
    df_validation_2 = df_train_copy.iloc[int(threshold * len(df_train)):, ]
    df_train_2 = df_train_copy.iloc[:int(threshold * len(df_train)), ]
    return df_validation_2.drop(['is_test_prob','Is_Test'], axis=1)#,df_train_copy.drop(['Is_Test'],axis=1)

In [ ]:
col_valid=cols.copy()
col_valid.append('label')
train_label[col_valid]

valid=对抗验证获取数据(train_label[col_valid].copy(),test_label[cols].copy(),0.6)

In [ ]:
def cv_model(clf, train_x, train_y, test_x, clf_name,valid_x):
    folds = 5
    seed = 2023
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
    
    oof = np.zeros(train_x.shape[0])
    predict = np.zeros(test_x.shape[0])
    predict_valid = np.zeros(valid_x.shape[0])
    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.01,
                'seed': 2020,
                'n_jobs':8
            }

            model = clf.train(params, train_matrix, 10000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[], verbose_eval=200, early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(test_x)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.05,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 8
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=10000, evals=watchlist, verbose_eval=1000, early_stopping_rounds=500)
            val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(test_matrix , ntree_limit=model.best_ntree_limit)
            
            valid_dk_matrix = clf.DMatrix(valid_x)
            valid_pred= model.predict(valid_dk_matrix, ntree_limit=model.best_ntree_limit)
        if clf_name == "cat":
            model = clf(
                        n_estimators=10000,
                        random_seed=2023,
                        eval_metric='AUC',
                        learning_rate=0.05,
                        max_depth=7,
                        early_stopping_rounds=200,
                        metric_period=500,
                    )

            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      use_best_model=True,
                      verbose=1)
            
            val_pred  = model.predict_proba(val_x)[:,1]
            test_pred = model.predict_proba(test_x)[:,1]
            valid_pred= model.predict_proba(valid_x)[:,1]
            
        oof[valid_index] = val_pred
        predict += test_pred / kf.n_splits
        predict_valid +=valid_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        print(cv_scores)
       
    return oof, predict,predict_valid,model

In [ ]:
xgb_oof, xgb_pred,xgb_valid_pred,clf = cv_model(CatBoostClassifier, train_label[cols], train_label['label'], test_label[cols], 'cat',valid.drop('label',axis=1))
#xgb_oof, xgb_pred,xgb_valid_pred,clf = cv_model(xgb, train_label[cols], train_label['label'], test_label[cols], 'xgb',valid.drop('label',axis=1))


In [ ]:
#计算shap值
import shap
explainer = shap.TreeExplainer(clf)

shap_values = explainer.shap_values(train_label[cols])
#shap.summary_plot(shap_values[1], train_label[cols])
shap.summary_plot(shap_values, train_label[cols])

In [ ]:
wrap_features=pd.DataFrame()
wrap_features['cols']=cols
wrap_features['shap']=abs(shap_values).sum(axis=0)
wrap_features.sort_values(by='shap',ascending=False,inplace=True)
wrap_features.index=range(len(wrap_features))
wrap_features=wrap_features[:200]['cols']

In [ ]:
#xgb_oof, xgb_pred,xgb_valid_pred,clf = cv_model(CatBoostClassifier, train_label[cols], train_label['label'], test_label[cols], 'cat',valid.drop('label',axis=1))
xgb_oof, xgb_pred,xgb_valid_pred,clf = cv_model(xgb, train_label[wrap_features], train_label['label'], test_label[wrap_features], 'xgb',valid[wrap_features])

In [ ]:
oof = xgb_oof
scores = []; thresholds = []
best_score = 0; best_threshold = 0

for threshold in np.arange(0.4,0.6,0.01):
    preds = (oof.reshape((-1))>threshold).astype('int')
    m = f1_score(train_label['label'].values.reshape((-1)), preds, average='macro')   
    scores.append(m)
    thresholds.append(threshold)
    if m>best_score:
        best_score = m
        best_threshold = threshold
    print(f'{threshold:.02f}, {m}')
print(f'最高分:{best_threshold:.2f}, {best_score}')

m_valid = f1_score(valid['label'].values, (xgb_valid_pred>best_threshold).astype('int'), average='macro')   

print(f'对抗验证分:{m_valid}')
# 0.47, 0.9150898680694286 # 0.86579572447
# 0.43, 0.9217716422203048 # 0.86697783
# 0.41, 0.9198568108353592 # 0.87674418605
# 0.40, 0.9231997065541027 # 0.87819025522
# 0.42, 0.913822737200522  # 0.87639132982 
# 0.40, 0.9148403872302214 # 0.88313184

In [ ]:
pred = xgb_pred
test_label['label'] = (pred.reshape((-1))>best_threshold).astype('int')
test_label[['账户代号','label']].to_csv('submission.csv', index=False)